In [1]:
import tensorflow
import os
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER

# data_utils.download_data_gdown("./") # gdrive-ckip
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

ws = WS("./data", disable_cuda=False)
pos = POS("./data", disable_cuda=False)
ner = NER("./data", disable_cuda=False)

C:\Users\ina\anaconda3\lib\site-packages\ckiptagger\model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
C:\Users\ina\anaconda3\lib\site-packages\keras\layers\legacy_rnn\rnn_cell_impl.py:984: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self._kernel = self.add_variable(
C:\Users\ina\anaconda3\lib\site-packages\keras\layers\legacy_rnn\rnn_cell_impl.py:993: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self._bias = self.add_variable(
C:\Users\ina\anaconda3\lib\site-packages\ckiptagger\model_pos.py:56: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future v

In [2]:
import numpy as np
import math
import re
import pkuseg
import os
import torch
import pandas as pd
from ckiptagger import construct_dictionary

Path = './Keywords/'
### create a coerce dictionary by the pest list and the chem list
dfpest = pd.read_excel(Path+'02pest.list.xlsx', engine='openpyxl', header=None)
dfchem = pd.read_excel(Path+'02chem.list.xlsx', engine='openpyxl', header=None)

coerce_dict = pd.concat([dfpest, dfchem]).to_numpy().reshape(-1).tolist()
coerce_dict = construct_dictionary(
    dict((word, 1) for word in coerce_dict if pd.isnull(word) == False)
    )

### create a recommend dictionary only by the crop list
dfcrop = pd.read_excel(Path+'02crop.list.xlsx', engine='openpyxl', header=None)
recommend_dict = dfcrop.to_numpy().reshape(-1).tolist()
recommend_dict = construct_dictionary(
    dict((word, 1) for word in recommend_dict if pd.isnull(word) == False)
    )

dftotal = pd.concat([dfpest, dfchem, dfcrop], ignore_index=True)

dftotal.insert(len(dftotal.columns), 'nan', None)
syms_dict = {}
for i in range(len(dftotal[0])):
    j = 0
    while (not pd.isnull(dftotal.iloc[i,j])):
        syms_dict[dftotal.iloc[i,j]] = dftotal.iloc[i,0]
        j += 1


In [3]:
textlist = [[],[]]
artilist = [[],[]]

for ind, DATATYPE in enumerate(['Public', 'Train']):
    Path = './data'+DATATYPE+'Complete/'
    filelist = os.listdir(Path)
    for file in filelist:
        f = open(Path+file, 'r', encoding='utf-8')
        article = f.read()
        artilist[ind].append(re.sub('[^\u4e00-\u9fa5]','', article))
    #     artilist.append(re.sub('[0-9]+|\x7f|\n|\!|\'|\?|\）|\（|\、|\。','', f.read()))
        text = ws([article], recommend_dictionary=recommend_dict, coerce_dictionary=coerce_dict)[0]
#         print(np.shape(text))
        textlist[ind].append([])
        for i in range(len(text)):
            if text[i] in syms_dict.keys():
                textlist[ind][-1].append(syms_dict[text[i]])
            elif  '\u4e00'<=text[i]<='\u9fff':
                textlist[ind][-1].append(text[i])
        f.close()


(490,)
(422,)
(57,)
(321,)
(520,)
(370,)
(461,)
(307,)
(177,)
(323,)
(226,)
(193,)
(191,)
(157,)
(168,)
(222,)
(180,)
(386,)
(235,)
(280,)
(208,)
(371,)
(215,)
(481,)
(302,)
(460,)
(358,)
(83,)
(378,)
(493,)
(268,)
(385,)
(166,)
(225,)
(231,)
(419,)
(351,)
(578,)
(523,)
(183,)
(433,)
(155,)
(313,)
(238,)
(356,)
(332,)
(546,)
(285,)
(469,)
(376,)
(318,)
(252,)
(436,)
(380,)
(359,)
(341,)
(294,)
(227,)
(342,)
(424,)
(417,)
(388,)
(332,)
(169,)
(272,)
(222,)
(245,)
(246,)
(225,)
(201,)
(233,)
(307,)
(217,)
(373,)
(554,)
(267,)
(243,)
(235,)
(286,)
(236,)
(224,)
(241,)
(172,)
(562,)
(169,)
(368,)
(201,)
(251,)
(217,)
(368,)
(563,)
(276,)
(726,)
(568,)
(229,)
(134,)
(725,)
(388,)
(667,)
(329,)
(669,)
(383,)
(167,)
(298,)
(462,)
(226,)
(249,)
(250,)
(208,)
(178,)
(347,)
(220,)
(566,)
(653,)
(318,)
(190,)
(640,)
(563,)
(302,)
(238,)
(317,)
(598,)
(186,)
(461,)
(252,)
(313,)
(602,)
(460,)
(423,)
(356,)
(535,)
(409,)
(522,)
(540,)
(424,)
(369,)
(483,)
(314,)
(417,)
(430,)
(391,)
(593,)
(318,)
(

In [4]:
dflabel = pd.read_csv('TrainLabel.csv')

labeltabel = np.zeros((len(filelist), len(filelist)))
intfilelist = [int(file[:-4]) for file in filelist]
label_transfer_dict = dict(zip(intfilelist, np.arange(len(filelist))))
l_t_dict_inv = dict(zip(np.arange(len(filelist)), intfilelist))

for i in range(len(dflabel)):
    dim1 = label_transfer_dict[dflabel['Reference'][i]]
    dim2 = label_transfer_dict[dflabel['Test'][i]]
    labeltabel[dim1, dim2] = 1
print(labeltabel[0:10,0:10])
print(l_t_dict_inv)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 1. 0. 1. 0. 1.]
 [0. 0. 1. 0. 1. 1. 0. 1. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 1. 1. 0. 1. 0. 0.]]
{0: 1, 1: 10, 2: 1000, 3: 1005, 4: 1007, 5: 1010, 6: 1011, 7: 1015, 8: 1016, 9: 1023, 10: 1025, 11: 103, 12: 1033, 13: 1037, 14: 1039, 15: 104, 16: 1041, 17: 1049, 18: 105, 19: 1052, 20: 1054, 21: 1056, 22: 1066, 23: 1067, 24: 1068, 25: 107, 26: 1073, 27: 1074, 28: 1075, 29: 1076, 30: 108, 31: 1082, 32: 1086, 33: 1087, 34: 1088, 35: 1093, 36: 110, 37: 1103, 38: 1111, 39: 1114, 40: 1116, 41: 1118, 42: 1119, 43: 1120, 44: 1125, 45: 1126, 46: 1127, 47: 1128, 48: 113, 49: 1130, 50: 1132, 51: 1133, 52: 1141, 53: 1142, 54: 1146, 55: 1147, 56: 1148, 57: 1151, 58: 1153, 59: 1155, 60: 1157, 61: 1159, 62: 1161, 63: 1165, 64: 1166, 65: 1167, 66: 117, 67: 1170, 68: 1172, 6

In [5]:
for ind, DATATYPE in enumerate(['Public', 'Train']):
    filelist = os.listdir('./data'+DATATYPE+'Complete/')
    if DATATYPE=='Train':
        for i in range(len(filelist)):
            np.savez('./'+DATATYPE+'_only/'+str(l_t_dict_inv[i]).zfill(4), 
                         text=textlist[ind][i], article=artilist[ind][i], label=labeltabel[i])
    elif DATATYPE=='Public':
        for i in range(len(filelist)):
            np.savez('./'+DATATYPE+'_only/'+(filelist[i][:-4]).zfill(4), 
                         text=textlist[ind][i], article=artilist[ind][i])

In [7]:
import pandas as pd
from gensim.models.word2vec import Word2Vec

word2vec_data = textlist[0]+textlist[1]
size = 50
epoch = 10

model = Word2Vec(word2vec_data,     
                    min_count=1,
                    vector_size=size,
                    epochs=epoch,
                    window=5,
                    sg=1,
                    seed=42,
                    batch_words=100
                )
def most_similar(w2v_model, words, topn=10):
    similar_df = pd.DataFrame()
    for word in words:
        try:
            similar_words = pd.DataFrame(w2v_model.wv.most_similar(word, topn=topn), columns=[word, 'cos'])
            similar_df = pd.concat([similar_df, similar_words], axis=1)
        except:
            print(word, "not found in Word2Vec model!")
    return similar_df

print(most_similar(model, ['稻熱病']))

model.save('w2v'+str(size)+'.model')
model1 = Word2Vec.load('w2v'+str(size)+'.model')

     稻熱病       cos
0   葉稻熱病  0.824533
1   穗稻熱病  0.724060
2    流行病  0.702776
3  葉鞘腐敗病  0.691118
4     流行  0.672593
5    紋枯病  0.663028
6     原有  0.652975
7     本病  0.642221
8     穗頸  0.637232
9    不稔粒  0.631621
